In [9]:
# ==========================================
# EXPERIMENT stage 3 – Test models with data augmentation (RUS)
# ==========================================

from datetime import date

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

from imblearn.under_sampling import RandomUnderSampler

# MODELS
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [10]:
# Load data from CSV file
df = pd.read_csv("creditcard_dataset.csv")
print("Rozkład klas oryginalny:", Counter(df['Class']))

Rozkład klas oryginalny: Counter({0: 284315, 1: 492})


In [11]:
# Data separation to features and labels
X = df.drop('Class', axis=1)
y = df['Class']

In [12]:
# Splitting data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Rozkład klas w zbiorze treningowym:", Counter(y_train))

Rozkład klas w zbiorze treningowym: Counter({0: 227451, 1: 394})


In [13]:
# Random Undersampling Implementation, RUS
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X, y)

print("Rozkład klas po undersamplingu RUS:", Counter(y_train_res))

Rozkład klas po undersamplingu RUS: Counter({0: 492, 1: 492})


In [14]:
# Models definition
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    "XGBoost": XGBClassifier(eval_metric='logloss', random_state=42, n_jobs=-1),
    "Logistic Regression": LogisticRegression(max_iter=5000, class_weight='balanced', random_state=42, solver='lbfgs'),
    "Linear SVC": LinearSVC(class_weight='balanced', max_iter=2000, random_state=42)
}

In [15]:
# Training and model validation after RUS
results = {}

for name, model in models.items():
    print(f"\nModel: {name}")
    model.fit(X_train_res, y_train_res)
    y_pred = model.predict(X_test)

    print(f"=== Results for model {name} ===")
    print(confusion_matrix(y_test, y_pred))
    report = classification_report(y_test, y_pred, target_names=["Correct", "Frauds"])
    print(report)
    
    results[name] = report


Model: Random Forest
=== Results for model Random Forest ===
[[55465  1399]
 [    0    98]]
              precision    recall  f1-score   support

     Correct       1.00      0.98      0.99     56864
      Frauds       0.07      1.00      0.12        98

    accuracy                           0.98     56962
   macro avg       0.53      0.99      0.56     56962
weighted avg       1.00      0.98      0.99     56962


Model: XGBoost
=== Results for model XGBoost ===
[[54929  1935]
 [    0    98]]
              precision    recall  f1-score   support

     Correct       1.00      0.97      0.98     56864
      Frauds       0.05      1.00      0.09        98

    accuracy                           0.97     56962
   macro avg       0.52      0.98      0.54     56962
weighted avg       1.00      0.97      0.98     56962


Model: Logistic Regression
=== Results for model Logistic Regression ===
[[55027  1837]
 [    8    90]]
              precision    recall  f1-score   support

     Correct

c:\Users\lzloc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
# Saving results to file
dataset_description = "RUS"
today = date.today()
file_name = "v3_results_" + dataset_description + ".txt"
with open(file_name, "w", encoding="utf-8") as f:
    f.write(f"Report created at : {today}\n")
    f.write(f"Comment           : RUS augmentation\n")
    f.write(f"Rozkład klas w zbiorze treningowym (RUS): {Counter(y_train_res)}\n")
    for name, report in results.items():
        f.write(f"\n=== {name} ===\n")
        f.write(report)
        f.write("\n" + "="*60 + "\n")

print("\n Experiment with dataset: " + dataset_description + " finished - see results in file: " + file_name)


 Experiment with dataset: RUS finished - see results in file: v3_results_RUS.txt
